In [1]:
import re
import pandas as pd
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pd.options.plotting.backend = "plotly"

In [5]:
cc = pd.read_csv('..\\data\\452\\DVT452_clamps.csv', usecols=[0,1,2,4,10])
cc['name'] = [re.sub(r'\s+', '', name) for name in cc['name']] # remove whitespace
cc.rename(columns={'perfAngleFromHighSideClockwiseDegrees': 'AngleFromHighSideClockwiseDegrees'}, inplace=True)
cc['type'] = ['CableDetectionClamp' if 'CDC' in name else 'CrossCouplingProtector' if 'CCP' in name else 'Centralizer' if 'CENT' in name else 'Other' for name in cc['name']]
cc['hadware_name'] = [name.split('.')[0] for name in cc['name']]
cc['obs'] = [name.split('-')[1] if '-' in name else pd.NA for name in cc['name']]

#cc = cc.loc[cc['type'] == 'OrientationClamp']

perfs = pd.read_csv('..\\data\\452\\DVT510_quickflags.csv', usecols=[4,0,10])
perfs['type'] = 'PERFS'
perfs.rename(columns={'perfAngleFromHighSideClockwiseDegrees': 'AngleFromHighSideClockwiseDegrees'}, inplace=True)

In [ ]:
cc_k = cc.loc[cc['passName'] == 'mainpass_up2_autogain']
cc = cc.loc[cc['passName'] == 'mainpass_up3_autogain']

In [ ]:
#deleting nv's
cc = cc.loc[cc['obs'] != 'NV']
cc_nv = cc.loc[cc['obs'] == 'NV']

In [6]:
features = cc[['type', 'hadware_name', 'depth', 'AngleFromHighSideClockwiseDegrees']].groupby(['type', 'hadware_name']).median(numeric_only=True)
features = features.sort_values(by=['depth'], ascending=False).reset_index()

features['fiberAngleFromHighSideClockwiseDegrees'] = [angle+180 if angle < 180 else angle-180 for angle in features['AngleFromHighSideClockwiseDegrees']] # plot angle for 180deg

features['fiberAngleFromHighSideClockwiseDegrees'] = [angle+90 if angle < 10 and ty=='Centralizer' else angle-90 if ty=='Centralizer' else angle for angle, ty in features[['fiberAngleFromHighSideClockwiseDegrees', 'type']].values]  # plot angle for 90deg

features['fiber_plot_angle'] = [angle-360 if angle > 180 else angle for angle in features['fiberAngleFromHighSideClockwiseDegrees']] # plot angle for 90deg


combined = pd.concat([features, perfs])
#combined = features
combined['plot_angle'] = [angle-360 if angle > 180 else angle for angle in combined['AngleFromHighSideClockwiseDegrees']] # plot angle for rest
combined['angle_rounded'] = combined['AngleFromHighSideClockwiseDegrees'].round(0)
combined['fiber_angle_rounded'] = combined['fiberAngleFromHighSideClockwiseDegrees'].round(0)

In [13]:
#convert from m to ft
combined['depth'] = combined['depth']*3.28084

In [17]:
from re import template


fig = go.Figure()

# Add windows
gozone_svg = ''.join([f'M {xy[0][0]+10},{xy[0][1]} ' if xy[1]==0 else f'L{xy[0][0]+10},{xy[0][1]} ' for xy in zip(combined.loc[combined['type']=='CableDetectionClamp', ['plot_angle','depth']].values,range(combined.loc[combined['type']=='CableDetectionClamp', ['plot_angle','depth']].shape[0]))])+''.join([f' L{xy[0][0]-10},{xy[0][1]} Z' if xy[1]==0 else f' L{xy[0][0]-10},{xy[0][1]}' for xy in zip(combined.loc[combined['type']=='CableDetectionClamp', ['plot_angle','depth']].values,range(combined.loc[combined['type']=='CableDetectionClamp', ['plot_angle','depth']].shape[0]))][::-1])

nogozone_svg = ''.join([f'M {xy[0][0]+10},{xy[0][1]} ' if xy[1]==0 else f'L{xy[0][0]+10},{xy[0][1]} ' for xy in zip(combined.loc[combined['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].values,range(combined.loc[combined['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].shape[0]))])+''.join([f' L{xy[0][0]-10},{xy[0][1]} Z' if xy[1]==0 else f' L{xy[0][0]-10},{xy[0][1]}' for xy in zip(combined.loc[combined['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].values,range(combined.loc[combined['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].shape[0]))][::-1])

fig.update_layout(shapes=[dict(type="path", path=gozone_svg, fillcolor='rgba(154,205,50,0.2)', line=dict(width=0), layer='below'), dict(type="path", path=nogozone_svg, fillcolor='rgba(255,69,0,0.2)', line=dict(width=0), layer='below')])

# Add traces
fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='CableDetectionClamp', 'plot_angle'], y=combined.loc[combined['type']=='CableDetectionClamp', 'depth'], mode='markers', name='CableDetectionClamp', marker_color='forestgreen', customdata=combined.loc[combined['type']=='CableDetectionClamp', ['hadware_name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='CrossCouplingProtector', 'plot_angle'], y=combined.loc[combined['type']=='CrossCouplingProtector', 'depth'], mode='markers', name='CrossCouplingProtector', marker_color='darkolivegreen', customdata=combined.loc[combined['type']=='CrossCouplingProtector', ['hadware_name', 'angle_rounded']]))

# fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='Centralizer', 'plot_angle'], y=combined.loc[combined['type']=='Centralizer', 'depth'], mode='markers', name='Centralizer', marker_color='cadetblue', customdata=combined.loc[combined['type']=='Centralizer', ['hadware_name', 'angle_rounded']]))

# fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='Other', 'plot_angle'], y=combined.loc[combined['type']=='Other', 'depth'], mode='markers', name='Other', marker_color='navy', customdata=combined.loc[combined['type']=='Other', ['hadware_name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='PERFS', 'plot_angle'], y=combined.loc[combined['type']=='PERFS', 'depth'], mode='markers', name='PERFS', marker_color='navy', customdata=combined.loc[combined['type']=='PERFS', ['name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='CableDetectionClamp', 'fiber_plot_angle'], y=combined.loc[combined['type']=='CableDetectionClamp', 'depth'], mode='lines+markers', name='Fiber Wire', marker_color='crimson', customdata=combined.loc[combined['type']=='CableDetectionClamp', ['hadware_name', 'fiber_angle_rounded']]))

fig.update_traces(hovertemplate='%{customdata[0]}<br>%{customdata[1]}')
fig.update_xaxes(range=[-185, 185])
fig.update_yaxes(autorange="reversed")
fig.update_layout(hovermode="y", title="Fiber cable orientation main3", legend_title="Type", legend_orientation="h", yaxis_title="Depth", xaxis_title='AngleFromHighSideClockwiseDegrees', height=2000, template="plotly")
fig.show()

In [ ]:
features['type'].value_counts()

In [ ]:
features['type'].value_counts()
O, CCP, C, CDC= features['type'].value_counts()

pie = make_subplots(rows=1, cols=1, specs=[[{"type": "domain"}]])

pie.add_trace(go.Pie(
     values=[CDC, CCP, C, O],
     labels=['CableDetectionClamp', 'CrossCouplingProtector', 'Centralizer', 'Other'],
     domain=dict(x=[0, 0.5]),
     name="Blast Protector",
     hoverinfo='label+value',
     sort=False,
     marker_colors = ['forestgreen', 'darkolivegreen', 'cadetblue', 'navy']
     ), 
     row=1, col=1)

pie.update_layout(title_text="Jewelry type", template="plotly")
pie.show()

In [ ]:
pio.write_html(fig, file='plotly/452_plot_main3.html', auto_open=False)
pio.write_html(pie, file='plotly/452_pie.html', auto_open=False)

kay plot

In [ ]:
features_k = cc_k[['type', 'hadware_name', 'depth', 'AngleFromHighSideClockwiseDegrees']].groupby(['type', 'hadware_name']).median(numeric_only=True)
features_k = features_k.sort_values(by=['depth'], ascending=False).reset_index()
features_k['fiberAngleFromHighSideClockwiseDegrees'] = [angle+180 if angle < 180 else angle-180 for angle in features_k['AngleFromHighSideClockwiseDegrees']]
features_k['fiber_plot_angle'] = [angle-360 if angle > 180 else angle for angle in features_k['fiberAngleFromHighSideClockwiseDegrees']]


# combined = pd.concat([features_k, perfs])
combined_k = features_k
combined_k['plot_angle'] = [angle-360 if angle > 180 else angle for angle in combined_k['AngleFromHighSideClockwiseDegrees']]
combined_k['angle_rounded'] = combined_k['AngleFromHighSideClockwiseDegrees'].round(0)
combined_k['fiber_angle_rounded'] = combined_k['fiberAngleFromHighSideClockwiseDegrees'].round(0)

In [ ]:
fig = go.Figure()

# Add windows
gozone_svg = ''.join([f'M {xy[0][0]+30},{xy[0][1]} ' if xy[1]==0 else f'L{xy[0][0]+30},{xy[0][1]} ' for xy in zip(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['plot_angle','depth']].values,range(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['plot_angle','depth']].shape[0]))])+''.join([f' L{xy[0][0]-30},{xy[0][1]} Z' if xy[1]==0 else f' L{xy[0][0]-30},{xy[0][1]}' for xy in zip(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['plot_angle','depth']].values,range(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['plot_angle','depth']].shape[0]))][::-1])

nogozone_svg = ''.join([f'M {xy[0][0]+10},{xy[0][1]} ' if xy[1]==0 else f'L{xy[0][0]+10},{xy[0][1]} ' for xy in zip(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].values,range(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].shape[0]))])+''.join([f' L{xy[0][0]-10},{xy[0][1]} Z' if xy[1]==0 else f' L{xy[0][0]-10},{xy[0][1]}' for xy in zip(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].values,range(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].shape[0]))][::-1])

fig.update_layout(shapes=[dict(type="path", path=gozone_svg, fillcolor='rgba(154,205,50,0.2)', line=dict(width=0), layer='below'), dict(type="path", path=nogozone_svg, fillcolor='rgba(255,69,0,0.2)', line=dict(width=0), layer='below')])

# Add traces
fig.add_trace(go.Scatter(x=combined_k.loc[combined_k['type']=='CableDetectionClamp', 'plot_angle'], y=combined_k.loc[combined_k['type']=='CableDetectionClamp', 'depth'], mode='markers', name='CableDetectionClamp', marker_color='forestgreen', customdata=combined_k.loc[combined_k['type']=='CableDetectionClamp', ['hadware_name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined_k.loc[combined_k['type']=='CrossCouplingProtector', 'plot_angle'], y=combined_k.loc[combined_k['type']=='CrossCouplingProtector', 'depth'], mode='markers', name='CrossCouplingProtector', marker_color='darkolivegreen', customdata=combined_k.loc[combined_k['type']=='CrossCouplingProtector', ['hadware_name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined_k.loc[combined_k['type']=='Centralizer', 'plot_angle'], y=combined_k.loc[combined_k['type']=='Centralizer', 'depth'], mode='markers', name='Centralizer', marker_color='cadetblue', customdata=combined_k.loc[combined_k['type']=='Centralizer', ['hadware_name', 'angle_rounded']]))

# fig.add_trace(go.Scatter(x=combined_k.loc[combined_k['type']=='PERFS', 'plot_angle'], y=combined_k.loc[combined_k['type']=='PERFS', 'depth'], mode='markers', name='PERFS', marker_color='navy', customdata=combined_k.loc[combined_k['type']=='PERFS', ['name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined_k.loc[combined_k['type']=='CableDetectionClamp', 'fiber_plot_angle'], y=combined_k.loc[combined_k['type']=='CableDetectionClamp', 'depth'], mode='lines+markers', name='Fiber Wire', marker_color='crimson', customdata=combined_k.loc[combined_k['type']=='CableDetectionClamp', ['hadware_name', 'fiber_angle_rounded']]))

fig.update_traces(hovertemplate='%{customdata[0]}<br>%{customdata[1]}')
fig.update_xaxes(range=[-185, 185])
fig.update_yaxes(autorange="reversed")
fig.update_layout(hovermode="y", title="Fiber cable orientation main2", legend_title="Type", legend_orientation="h", template="plotly", yaxis_title="Depth", xaxis_title='AngleFromHighSideClockwiseDegrees')
fig.show()

In [ ]:
pio.write_html(fig, file='plotly/445_plot_main2.html', auto_open=False)
pio.write_html(pie, file='plotly/445_pie.html', auto_open=False)